In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Define the dataset
data = open('/kaggle/input/poem-generation/poem.txt', encoding="utf8").read()

# Tokenization and Padding for Poetry Generation
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Model Architecture for Poetry Generation
poetry_model = Sequential()
poetry_model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
poetry_model.add(Bidirectional(LSTM(150)))
poetry_model.add(Dropout(0.2))
poetry_model.add(Dense(total_words, activation='softmax'))
poetry_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training for Poetry Generation
poetry_model.fit(X, y, epochs=100, verbose=1)

# Example of Poetry Generation
def generate_poem(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(poetry_model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

seed_text = "And"
generated_poem = generate_poem(seed_text, 10, max_sequence_len)
print("Generated Poem:", generated_poem)


Epoch 1/100
510/510 [==============================] - 28s 47ms/step - loss: 6.9182 - accuracy: 0.0630
Epoch 2/100
510/510 [==============================] - 24s 47ms/step - loss: 6.4735 - accuracy: 0.0740
Epoch 3/100
510/510 [==============================] - 24s 46ms/step - loss: 6.2215 - accuracy: 0.0833
Epoch 4/100
510/510 [==============================] - 24s 47ms/step - loss: 5.9607 - accuracy: 0.0961
Epoch 5/100
510/510 [==============================] - 23s 46ms/step - loss: 5.6746 - accuracy: 0.1080
Epoch 6/100
510/510 [==============================] - 23s 46ms/step - loss: 5.3604 - accuracy: 0.1215
Epoch 7/100
510/510 [==============================] - 27s 53ms/step - loss: 5.0384 - accuracy: 0.1365
Epoch 8/100
510/510 [==============================] - 26s 51ms/step - loss: 4.7001 - accuracy: 0.1567
Epoch 9/100
510/510 [==============================] - 26s 52ms/step - loss: 4.3543 - accuracy: 0.1866
Epoch 10/100
510/510 [==============================] - 25s 49ms/step - l

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display

# Define the dataset
data = open('/kaggle/input/poem-generation/poem.txt', encoding="utf8").read()

# Tokenization and Padding for Poetry Generation
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Model Architecture for Poetry Generation
poetry_model = Sequential()
poetry_model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
poetry_model.add(Bidirectional(LSTM(150)))
poetry_model.add(Dropout(0.2))
poetry_model.add(Dense(total_words, activation='softmax'))
poetry_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training for Poetry Generation
poetry_model.fit(X, y, epochs=100, verbose=1)

# Function to generate poem
def generate_poem(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(poetry_model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# User input widget for poem generation
poem_input = widgets.Text(placeholder='Enter your seed text for poem generation')
display(poem_input)

# Button to trigger poem generation
poem_button = widgets.Button(description='Generate Poem')
display(poem_button)

# Output widget for displaying generated poem
poem_output = widgets.Output()
display(poem_output)

def generate_poem_output(b):
    with poem_output:
        poem_output.clear_output()
        seed_text = poem_input.value.strip()
        if seed_text:
            generated_poem = generate_poem(seed_text, 10, max_sequence_len)
            print("Generated Poem:")
            print(generated_poem)
        else:
            print("Please enter a seed text.")

poem_button.on_click(generate_poem_output)

# Function to perform sentiment analysis
def predict_sentiment(text):
    # Perform sentiment analysis here (replace with your sentiment analysis model)
    # For demonstration, returning a random sentiment label
    sentiments = ['positive', 'neutral', 'negative']
    return np.random.choice(sentiments)

# User input widget for sentiment analysis
sentiment_input = widgets.Text(placeholder='Enter your text for sentiment analysis')
display(sentiment_input)

# Button to trigger sentiment analysis
sentiment_button = widgets.Button(description='Analyze Sentiment')
display(sentiment_button)

# Output widget for displaying sentiment analysis result
sentiment_output = widgets.Output()
display(sentiment_output)

def analyze_sentiment(b):
    with sentiment_output:
        sentiment_output.clear_output()
        input_text = sentiment_input.value.strip()
        if input_text:
            sentiment = predict_sentiment(input_text)
            print("Sentiment:", sentiment)
        else:
            print("Please enter some text for sentiment analysis.")

sentiment_button.on_click(analyze_sentiment)


Epoch 1/100
510/510 [==============================] - 29s 48ms/step - loss: 6.9309 - accuracy: 0.0622
Epoch 2/100
510/510 [==============================] - 25s 49ms/step - loss: 6.4867 - accuracy: 0.0735
Epoch 3/100
510/510 [==============================] - 24s 46ms/step - loss: 6.2337 - accuracy: 0.0828
Epoch 4/100
510/510 [==============================] - 24s 48ms/step - loss: 5.9488 - accuracy: 0.0982
Epoch 5/100
510/510 [==============================] - 24s 47ms/step - loss: 5.6433 - accuracy: 0.1088
Epoch 6/100
510/510 [==============================] - 24s 47ms/step - loss: 5.3115 - accuracy: 0.1245
Epoch 7/100
510/510 [==============================] - 23s 46ms/step - loss: 4.9807 - accuracy: 0.1435
Epoch 8/100
510/510 [==============================] - 24s 47ms/step - loss: 4.6397 - accuracy: 0.1634
Epoch 9/100
510/510 [==============================] - 24s 46ms/step - loss: 4.2833 - accuracy: 0.1936
Epoch 10/100
510/510 [==============================] - 23s 45ms/step - l

Text(value='', placeholder='Enter your seed text for poem generation')

Button(description='Generate Poem', style=ButtonStyle())

Output()

Text(value='', placeholder='Enter your text for sentiment analysis')

Button(description='Analyze Sentiment', style=ButtonStyle())

Output()